In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import RidgeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                            balanced_accuracy_score, confusion_matrix,
                            matthews_corrcoef, cohen_kappa_score)
from sklearn.impute import SimpleImputer
import time
import os

# Create directory for visualizations
os.makedirs("confusion_matrices", exist_ok=True)
os.makedirs("visualizations", exist_ok=True)
os.makedirs("results", exist_ok=True)

# Set the number of K folds
K_FOLDS = 2

# Helper function for confusion matrix metrics
def confusion_matrix_metrics(cm, classes):
    metrics = {}
    for idx, class_label in enumerate(classes):
        TP = cm[idx, idx]  # True Positives for this class
        FP = cm[:, idx].sum() - TP  # False Positives for this class
        FN = cm[idx, :].sum() - TP  # False Negatives for this class
        TN = cm.sum() - (TP + FP + FN)  # True Negatives for this class

        metrics[class_label] = {
            'TPR': TP / (TP + FN + 1e-10) if (TP + FN) > 0 else 0,
            'TNR': TN / (TN + FP + 1e-10) if (TN + FP) > 0 else 0,
            'FPR': FP / (FP + TN + 1e-10) if (FP + TN) > 0 else 0,
            'FNR': FN / (FN + TP + 1e-10) if (FN + TP) > 0 else 0
        }
    return metrics

# Load dataset
print("Loading dataset...")
try:
    df = pd.read_csv('C:/Users/ddihora1604/Downloads/IIT Patna/Task/Dataset 2/part-00001_preprocessed_dataset.csv')
    print(f"Dataset loaded successfully with shape: {df.shape}")
except Exception as e:
    print(f"Error loading dataset: {e}")
    exit(1)

# Take 20% of the data for faster processing (if needed)
df = df.sample(frac=0.2, random_state=42)
print(f"Sampled dataset shape: {df.shape}")

# Rename the last column as 'label' if not already named
df.rename(columns={df.columns[-1]: 'label'}, inplace=True)

# Data preprocessing
print("Preprocessing data...")

# Handle missing values
imputer = SimpleImputer(strategy='mean')
X = df.drop(columns=['label'])
X_columns = X.columns
X = imputer.fit_transform(X)
y = df['label'].values

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Initialize results storage
results = []
timing_results = []

# Set up KFold cross-validation
kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

# Ridge classifier with hyperparameters (removed deprecated 'normalize' parameter)
ridge_classifier = RidgeClassifier(
    alpha=1.0,  # Regularization strength
    fit_intercept=True,
    copy_X=True,
    max_iter=1000,
    tol=0.001,
    class_weight=None,
    solver='auto',
    random_state=42
)

# Cross-validation process
print(f"Starting {K_FOLDS}-fold cross-validation with Ridge Classifier...")
fold_idx = 1
for train_index, test_index in kf.split(X):
    print(f"Processing fold {fold_idx}/{K_FOLDS}...")
    
    # Split the data for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Record training time
    start_train_time = time.time()
    ridge_classifier.fit(X_train, y_train)
    train_time = time.time() - start_train_time
    
    # Record prediction time
    start_test_time = time.time()
    y_pred = ridge_classifier.predict(X_test)
    test_time = time.time() - start_test_time
    
    # Record timing info
    timing_results.append({
        'Classifier': 'RidgeClassifier',
        'Fold': fold_idx,
        'Training Time (s)': train_time,
        'Testing Time (s)': test_time,
        'Total Time (s)': train_time + test_time
    })
    
    # Compute evaluation metrics
    unique_classes = np.unique(y)
    cm = confusion_matrix(y_test, y_pred, labels=unique_classes)
    cm_metrics = confusion_matrix_metrics(cm, unique_classes)
    
    # Calculate metrics for each class
    for class_label in unique_classes:
        # Binary classification metrics for this class
        y_test_binary = (y_test == class_label).astype(int)
        y_pred_binary = (y_pred == class_label).astype(int)
        
        # Calculate metrics
        class_metrics = {
            'Classifier': 'RidgeClassifier',
            'Fold': fold_idx,
            'Class': class_label,
            'Accuracy': accuracy_score(y_test_binary, y_pred_binary),
            'Precision': precision_score(y_test_binary, y_pred_binary, zero_division=0),
            'Recall': recall_score(y_test_binary, y_pred_binary),
            'F1 Score': f1_score(y_test_binary, y_pred_binary),
            'Balanced Accuracy': balanced_accuracy_score(y_test_binary, y_pred_binary),
            'Matthews Correlation Coefficient': matthews_corrcoef(y_test_binary, y_pred_binary),
            'Cohen Kappa Score': cohen_kappa_score(y_test_binary, y_pred_binary),
            'True Positive Rate (TPR)': cm_metrics[class_label]['TPR'],
            'True Negative Rate (TNR)': cm_metrics[class_label]['TNR'],
            'False Positive Rate (FPR)': cm_metrics[class_label]['FPR'],
            'False Negative Rate (FNR)': cm_metrics[class_label]['FNR'],
            'Training Time (s)': train_time,
            'Testing Time (s)': test_time
        }
        
        results.append(class_metrics)
    
    # Plot and save confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=unique_classes, yticklabels=unique_classes)
    plt.title(f"Ridge Classifier - Fold {fold_idx} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(f"confusion_matrices/fold_{fold_idx}.png")
    plt.close()
    
    # Feature importance visualization (coefficients of Ridge Classifier)
    if fold_idx == 1:  # Only for the first fold to avoid redundancy
        coef = ridge_classifier.coef_
        # For multiclass, average the absolute coefficients across classes
        if coef.ndim > 1:
            importance = np.mean(np.abs(coef), axis=0)
        else:
            importance = np.abs(coef)
        
        # Plot feature importance
        plt.figure(figsize=(12, 8))
        feature_importance = pd.DataFrame({
            'Feature': X_columns,
            'Importance': importance
        })
        feature_importance = feature_importance.sort_values('Importance', ascending=False)
        
        # Plot top 20 features or all if less than 20
        top_n = min(20, len(feature_importance))
        sns.barplot(x='Importance', y='Feature', data=feature_importance.head(top_n))
        plt.title('Ridge Classifier Feature Importance')
        plt.tight_layout()
        plt.savefig(f"visualizations/feature_importance.png")
        plt.close()
    
    # Pair plot for the first 5 features and target
    if fold_idx == 1:  # Only for the first fold
        # Convert numpy array back to DataFrame for visualization
        sample_size = min(1000, X_test.shape[0])  # Limit sample size for visualization
        vis_df = pd.DataFrame(X_test[:sample_size], columns=X_columns)
        vis_df['label'] = y_test[:sample_size]
        
        # Select first 5 features (or less if fewer features exist)
        plot_features = list(X_columns[:min(5, len(X_columns))])
        plot_features.append('label')
        
        # Create pair plot
        plt.figure(figsize=(12, 10))
        pair_plot = sns.pairplot(vis_df[plot_features], hue='label', height=2.5)
        plt.suptitle('Pair Plot of Top Features by Ridge Classifier', y=1.02)
        plt.tight_layout()
        pair_plot.savefig("visualizations/pair_plot.png")
        plt.close()
    
    fold_idx += 1

# Create DataFrames for results and save to CSV
timing_df = pd.DataFrame(timing_results)
timing_df.to_csv("results/timing.csv", index=False)

results_df = pd.DataFrame(results)
print("Classification Metrics Across Folds:")
print(results_df.head())

# Save results to CSV
results_df.to_csv("results/metrics.csv", index=False)

# Calculate and display average metrics across folds
avg_metrics = results_df.groupby(['Classifier', 'Class']).mean().reset_index()
avg_metrics.to_csv("results/avg_metrics.csv", index=False)
print("\nAverage Metrics Across Folds:")
print(avg_metrics[['Classifier', 'Class', 'Accuracy', 'Precision', 'Recall', 'F1 Score']])

print("\nRidge Classifier implementation completed successfully!")


Loading dataset...
Dataset loaded successfully with shape: (218805, 47)
Sampled dataset shape: (43761, 47)
Preprocessing data...
Starting 2-fold cross-validation with Ridge Classifier...
Processing fold 1/2...


c:\Users\ddihora1604\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ddihora1604\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ddihora1604\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Processing fold 2/2...


c:\Users\ddihora1604\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ddihora1604\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ddihora1604\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Classification Metrics Across Folds:
        Classifier  Fold  Class  Accuracy  Precision    Recall  F1 Score  \
0  RidgeClassifier     1      0  0.999954   0.000000  0.000000  0.000000   
1  RidgeClassifier     1      1  0.984416   0.644961  0.787879  0.709292   
2  RidgeClassifier     1      2  0.999954   0.000000  0.000000  0.000000   
3  RidgeClassifier     1      3  0.999863   0.000000  0.000000  0.000000   
4  RidgeClassifier     1      4  0.988575   0.310897  0.734848  0.436937   

   Balanced Accuracy  Matthews Correlation Coefficient  Cohen Kappa Score  \
0           0.500000                          0.000000           0.000000   
1           0.888577                          0.705078           0.701367   
2           0.500000                          0.000000           0.000000   
3           0.500000                          0.000000           0.000000   
4           0.862481                          0.473511           0.432122   

   True Positive Rate (TPR)  True Negative 

<Figure size 1200x1000 with 0 Axes>